# Input & Label

In [1]:
from rtmag.paper.load import load_input_label, MyModel

In [2]:
input_files, label_files = load_input_label("/mnt/f/isee_dataset/12673")
len(input_files), len(label_files)

(389, 389)

In [3]:
idx = 240
input_files[idx]

PosixPath('/mnt/f/isee_dataset/12673/input/input_12673_20170906_084800.npz')

# Model prediction & Label (NLFFF, Potential)

In [4]:
from time import time

In [5]:
# mm = MyModel("/home/mgjeon/space/workspace/base/pino_CC_Hnorm_Unit_2/best_model.pt")
mm = MyModel("/home/mgjeon/space/workspace/base/uno_pi_cc_hnorm_unit_aug/best_model.pt")
# mm2 = MyModel("/home/mgjeon/space/workspace/base/uno_pi_cc_hnorm_unit_aug/model_60.pt", clip=100000)
# mm3 = MyModel("/home/mgjeon/space/workspace/base/uno_pi_cc_hnorm_1_unit_aug_ccc_square/model_50.pt", clip=100000)

Model loaded from epoch 40


In [7]:
start_time = time()
b_prev = mm.get_pred(input_files[idx-1])
print(f"{time() - start_time} seconds")

0.47913527488708496 seconds


In [8]:
start_time = time()
b = mm.get_pred(input_files[idx])
print(f"{time() - start_time} seconds")

0.5142924785614014 seconds


In [9]:
start_time = time()
B_prev = mm.get_label(label_files[idx-1])
print(f"{time() - start_time} seconds")

1.7844133377075195 seconds


In [10]:
start_time = time()
B = mm.get_label(label_files[idx])
print(f"{time() - start_time} seconds")

1.7700204849243164 seconds


In [11]:
start_time = time()
Bp_prev = mm.get_pot(label_files[idx-1])
print(f"{time() - start_time} seconds")

1.8032095432281494 seconds


In [12]:
start_time = time()
Bp = mm.get_pot(label_files[idx])
print(f"{time() - start_time} seconds")

1.7851359844207764 seconds


In [17]:
start_time = time()
dx_prev, dy_prev, dz_prev, dV_prev = mm.get_dV(input_files[idx-1])
print(f"{time() - start_time} seconds")

0.003612041473388672 seconds


In [18]:
start_time = time()
dx, dy, dz, dV = mm.get_dV(input_files[idx])
print(f"{time() - start_time} seconds")

0.0058193206787109375 seconds


In [19]:
dV_prev, dV

(1.991674e+23, 1.6434918e+23)

# Energy

In [22]:
from rtmag.paper.metric import energy, energy_unit

In [24]:
start_time = time()
print("energy (pred):", energy_unit(b_prev))
print(f"{time() - start_time} seconds")

energy (pred): 324055252800.74994
0.11580133438110352 seconds


In [25]:
start_time = time()
print("energy (pred):", energy_unit(b))
print(f"{time() - start_time} seconds")

energy (pred): 424282459483.84955
0.21135354042053223 seconds


In [21]:
start_time = time()
print("energy (pred):", energy(b_prev, dV_prev)/1e33)
print(f"{time() - start_time} seconds")

energy (pred): 2.5680145092007276
0.28102779388427734 seconds


In [162]:
start_time = time()
print("energy (pred):", energy(b, dV)/1e33)
print(f"{time() - start_time} seconds")

energy (pred): 2.5999752259158146
0.1361064910888672 seconds


In [141]:
start_time = time()
print("energy (pred):", energy(b_prev, dV)/1e33)
print(f"{time() - start_time} seconds")

energy (pred): 3.3622769652304605
0.11423397064208984 seconds


In [142]:
start_time = time()
print("energy (pred):", energy(b, dV)/1e33)
print(f"{time() - start_time} seconds")

energy (pred): 2.592123464746674
0.10830807685852051 seconds


In [143]:
start_time = time()
print("energy (pred):", energy(b2_prev, dV)/1e33)
print(f"{time() - start_time} seconds")

energy (pred): 3.4040884274641026
0.10586237907409668 seconds


In [144]:
start_time = time()
print("energy (pred):", energy(b2, dV)/1e33)
print(f"{time() - start_time} seconds")

energy (pred): 2.6406863053305845
0.14602160453796387 seconds


In [145]:
start_time = time()
print("energy (pred):", energy(b3_prev, dV)/1e33)
print(f"{time() - start_time} seconds")

energy (pred): 3.3055285875642633
0.11481738090515137 seconds


In [146]:
start_time = time()
print("energy (pred):", energy(b3, dV)/1e33)
print(f"{time() - start_time} seconds")

energy (pred): 2.3594974946808387
0.11844348907470703 seconds


In [147]:
start_time = time()
print("energy (true):", energy(B_prev, dV)/1e33)
print(f"{time() - start_time} seconds")

energy (true): 3.0000347446223325
0.052926063537597656 seconds


In [148]:
start_time = time()
print("energy (true):", energy(B, dV)/1e33)
print(f"{time() - start_time} seconds")

energy (true): 2.3702460967001286
0.05226755142211914 seconds


In [149]:
start_time = time()
print("energy (true):", energy(Bp_prev, dV)/1e33)
print(f"{time() - start_time} seconds")

energy (true): 2.2195474587663084
0.0522768497467041 seconds


In [150]:
start_time = time()
print("energy (true):", energy(Bp, dV)/1e33)
print(f"{time() - start_time} seconds")

energy (true): 1.833690940560429
0.052191734313964844 seconds


# Quantitative Evaluation

In [19]:
import numpy as np
from rtmag.paper.metric import evaluate, evaluate_exclude, C_vec, C_cs, En_prime, Em_prime, eps

In [20]:
start_time = time()
res = evaluate(b, B)
print(f"{time() - start_time} seconds")

C_vec     : 0.94
C_cs      : 0.70
E_n'      : 0.42
E_m'      : -0.53
eps       : 1.12
1.042083501815796 seconds


In [21]:
start_time = time()
res = evaluate(b2, B)
print(f"{time() - start_time} seconds")

C_vec     : 0.94
C_cs      : 0.64
E_n'      : 0.38
E_m'      : -0.78
eps       : 1.13
1.0333340167999268 seconds


In [22]:
start_time = time()
res = evaluate(b3, B)
print(f"{time() - start_time} seconds")

C_vec     : 0.95
C_cs      : 0.74
E_n'      : 0.47
E_m'      : -0.37
eps       : 1.10
1.0917751789093018 seconds


In [23]:
start_time = time()
res = evaluate(b[:, :, :64, :], B[:, :, :64, :])
print(f"{time() - start_time} seconds")

C_vec     : 0.95
C_cs      : 0.78
E_n'      : 0.59
E_m'      : 0.25
eps       : 1.08
0.29225707054138184 seconds


In [24]:
start_time = time()
res = evaluate(b2[:, :, :64, :], B[:, :, :64, :])
print(f"{time() - start_time} seconds")

C_vec     : 0.94
C_cs      : 0.75
E_n'      : 0.56
E_m'      : 0.15
eps       : 1.10
0.30597352981567383 seconds


In [25]:
start_time = time()
res = evaluate(b3[:, :, :64, :], B[:, :, :64, :])
print(f"{time() - start_time} seconds")

C_vec     : 0.95
C_cs      : 0.79
E_n'      : 0.62
E_m'      : 0.33
eps       : 1.07
0.2962942123413086 seconds


In [26]:
start_time = time()
res = evaluate((0.5*(b+b2)), B)
print(f"{time() - start_time} seconds")

C_vec     : 0.94
C_cs      : 0.67
E_n'      : 0.41
E_m'      : -0.63
eps       : 1.12
1.2403695583343506 seconds


In [27]:
start_time = time()
res = evaluate((0.5*(b+b3)), B)
print(f"{time() - start_time} seconds")

C_vec     : 0.95
C_cs      : 0.72
E_n'      : 0.46
E_m'      : -0.42
eps       : 1.10
1.4300146102905273 seconds


In [28]:
start_time = time()
res = evaluate_exclude(b, B)
print(f"{time() - start_time} seconds")

C_vec     : 0.94
C_cs      : 0.70
E_n'      : 0.42
E_m'      : -0.53
eps       : 1.12
1.138474702835083 seconds


In [29]:
start_time = time()
print("C_vec:", C_vec(b, B))
print(f"{time() - start_time} seconds")

C_vec: 0.9410427068730733
0.23827719688415527 seconds


In [30]:
start_time = time()
print("C_cs:", C_cs(b, B))
print(f"{time() - start_time} seconds")

C_cs: 0.6976907722221578
0.2826650142669678 seconds


In [31]:
start_time = time()
print("En_prime:", En_prime(b, B))
print(f"{time() - start_time} seconds")

En_prime: 0.4237618450386049
0.18692898750305176 seconds


In [32]:
start_time = time()
print("Em_prime:", Em_prime(b, B))
print(f"{time() - start_time} seconds")

Em_prime: -0.5293472642153414
0.20438218116760254 seconds


In [33]:
start_time = time()
print("eps:", eps(b, B))
print(f"{time() - start_time} seconds")

eps: 1.1207460084445755
0.15120601654052734 seconds


In [27]:
start_time = time()
bf = mm.fine_tuning(dx=dx/1e8, dy=dy/1e8, dz=dz/1e8)
print(f"{time() - start_time} seconds")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
74.41732859611511 seconds


In [28]:
start_time = time()
res = evaluate(bf, B)
print(f"{time() - start_time} seconds")

C_vec     : 0.96
C_cs      : 0.72
E_n'      : 0.55
E_m'      : 0.17
eps       : 1.05
1.0380771160125732 seconds
